# NER

Fichier pris en entrée :
- clean_{date}.json -> provient du notebook **nettoyeur**

Fichiers de sortie produits par le notebook :
- NLP_{date}_{type_pipeline}_{nombre_resultats}.html -> **fichier HTML des résultats de la NER**

- NLP_{date}_{type_pipeline}_{nombre_resultats}.csv -> **tableau avec autant de lignes que de résultats et deux colonnes : titre et pers**
**-> pers** = entitées nommées "personnage" issues de spacy pour lesquelles un traitement de nettoyage a été effectué :
    - suppression des termes inférieur à 2 caractères
    - suppression des doublons malgré d'éventuelles coquilles d'OCR dans la graphie du nom -> la graphie retenue est celle qui revient le plus de fois au sein du CR

In [1]:
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
import fr_core_news_lg # md, sm
nlp = fr_core_news_lg.load()
from datetime import date
import time
import json
import itertools
import regex as re
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# Variables à changer par l'utilisateur
source_json = 'results/clean_w_names_24-01-23.json'
pipeline = "fr_core_news_lg" # /!\ PENSER À CHANGER PLUS HAUT

# Importation des données depuis le JSON
df = pd.read_json(source_json, orient='index')

# Nombre de résultats dans la réponse produite, varie en fonction des essais
# nb_a_traiter = len(df.index)
nb_a_traiter = 100

# Variable permettant de compter puis d'inscrire le nombre de "personnes" détéctées par la NER dans le nom du fichier de sortie final (pour contrôle)
count_names = 0

In [3]:
# Définition de l'en tête général
date = time.strftime("%d-%m-%y")

# (Re)création d'un fichier de sortie propre + en tête
nom_fichier = f"results/NLP_{date}_{pipeline[-2:]}_{nb_a_traiter}r_{count_names}n"

In [4]:
lists = []

# Boucle de constitution : export PER + HTML
for i in range(nb_a_traiter):
    # NLP par spacy --------------------------------------------------------------------------
    title = df.loc[i, 'area_title']
    text = df.loc[i, 'area_text']
    extracted_names = df.loc[i, 'extracted_names']
    doc = nlp(text)

    # Constitution de la grande liste destinée à être convertie en df ------------------------
    pers = [(ent.text) for ent in doc.ents if ent.label_ == "PER"]
    # list = [title, pers, extracted_names]
    list = [title, pers, pers]
    lists.append(list)

In [5]:
# Conversion de la liste vers df
df_PER = pd.DataFrame(lists, columns=['title', 'pers', 'extracted_names'])

In [ ]:
# Fonction de nettoyage de la liste des entitées reconnues
def clean_names(list):
    patterns = [
        r'^M\.',
        r'^MM\.',
        r'^MMe.',
        r'^MMe ',
        r'^ +',
    ]
    for i in range(len(list)):
        for pattern in patterns:
            list[i] = re.sub(pattern, '', list[i])
    return(list)

# Comptage des termes au sein de la liste
def make_count(list):
    list_count = []
    for i in range(len(list)):
        mot = list[i]
        count = list.count(list[i])
        list_count.append([mot, count])
    return(list_count)

# Fonction de comparaisons des mots 2 à 2 : lorsque plusieurs noms sont similaires, on n'en garde qu'un
# Fonction qui prend une liste en entrée et ressort une liste en sortie
def keep_best_name(list):
    discri_list = [
        r'^[\P{Lu}]',
        r'[0-9]+',
        r'\.$',
        r'\"|\'|\\|\/|«|»'
    ]

    # Pour chaque mot
    temp_list = []
    for i in range(len(list)):
        ref = list[i]

        # On va comparera avec tous autres mots de la liste
        for j in range(len(list)):
            # Si on confronte le mot avec lui même -> on ne fait rien
            if i == j:
                pass
            else:
                # Sinon comparaison du score entre les deux noms
                ratio = fuzz.token_sort_ratio(ref[0], list[j][0])
                partial_ratio = fuzz.partial_ratio(ref[0], list[j][0])
                if ratio > 85 or partial_ratio > 85:
                    # Si le second mot est présent plus de fois, on le retient lui
                    if ref[1] < list[j][1]:
                        ref = list[j]

        # Discrimination des noms de 2 lettres ou moins, ceux qui contiennent de chiffres, qui ne commencent pas par une maj et ceux ne contenant pas de voyelles
        if (ref[0] not in temp_list) & (len(ref[0]) > 2):
            ready = True
            for pattern in discri_list:
                if re.search(pattern, ref[0]):
                    ready = False
                if not re.search('(?i)[aeiouy]+', ref[0]):
                    ready = False
            if ready:
                temp_list.append(ref[0])
    return(temp_list)

In [ ]:
# Pour chaque ligne de notre tableau de départ
for i in range(len(df_PER.index)):
    # On appelle la fonction de nettoyage sur nos entitées "personnes"
    list_PER_clean = clean_names(df_PER['pers'][i])
    # Chaque entitée est compté, un nombre d'occurence lui est attribué = [['Personne1', 2], ['Personne2', 3], ['Personne3', 1], ['Personne4', 1]]
    list_PER_count = make_count(list_PER_clean)
    # De tous les noms qui se ressemblent (= coquilles dans la graphie), on ne garde que l'occurence qui est apparue le plus de fois
    best_names = keep_best_name(list_PER_count)
    # Décompte pour contrôle 
    count_names += len(best_names)
    # Màj de notre liste d'entités "personnes" au sein du dataframe
    df_PER.at[i, 'pers'] = best_names

In [ ]:
# (Re)création d'un fichier de sortie propre + en tête
nom_fichier = f"results/NLP_{date}_{pipeline[-2:]}_{nb_a_traiter}r_{count_names}n"

# Conversion du df vers csv
df_PER.to_csv(f"{nom_fichier}.csv")

count_names

In [ ]:
after = df_PER['pers'][0]
print(f'AVANT : {before}')
print('----------------------')
print(f'APRÈS : {after}')